# Numerical Integration of Functions

In science and engineering, many important quantities are obtained by
integrating functions.
Examples include the total energy radiated by a star (integral of
brightness over wavelength),
the probability of finding a quantum particle in a region (integral of
probability density), or
the synchrotron emissivity function (integral over electron
distribution function).

Analytical solutions are rare because real functions often come from
measurements, simulations, or complicated models.
In such cases, we rely on **numerical integration** (or *quadrature*)
to approximate
\begin{align}
  I = \int_a^b f(x)\,dx,
\end{align}
using only a finite number of function evaluations.

Numerical integration provides a controlled setting to study how
approximations are constructed, how accuracy depends on step size, and
how errors accumulate.
These lessons generalize directly to solving differential equations
and more complex simulations, making integration an essential starting
point for good numerical practice.

To study numerical integration systematically, it helps to begin with
a function whose integral we know exactly.
Consider $f(x) = e^x$.
Its definite integral from $a$ to $b$ is
\begin{align}
  I = \int_a^b e^x \, dx = e^b - e^a.
\end{align}
On the interval $[0,1]$, the exact value is $I = e - 1$.
This known result will allow us to check the accuracy of numerical
approximations as we vary step size and method.

Below is a simple plot of $f(x)=e^x$ on $[0,1]$ for visual reference.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    return np.exp(x)

# Define a fine grid for plotting
x = np.linspace(0, 1, 201)
y = f(x)

# Plot the function and shade the area under the curve
plt.plot(x, y, label=r"$f(x) = e^x$")
plt.fill_between(x, y, alpha=0.3)
plt.title(r'Function $f(x) = e^x$ on $[0,1]$')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()